In [ ]:
# DATA INGESTION
# !pip install langchain langchain-community
# ! pip install beautifulsoup4 requests
# ! pip install pypdf
# ! pip install chromadb
! pip install faiss-cpu

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("speech.txt")
text_document = loader.load()
text_document

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.getenv("OPENAI_API_KEY")



ModuleNotFoundError: No module named 'dotenv'

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

In [ ]:
text_documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('attention.pdf')
pdf_document = loader.load()
pdf_document

Splitting Text (Tranform)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap = 200)
documents = text_splitter.split_documents(pdf_document)

documents[:5]

In [ ]:
# Vector embeddings and vector store

from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

db= Chroma.from_documents(documents[:20],OpenAIEmbeddings())

In [ ]:
## Vector database
query = "An attention function can be?"
result = db.similarity_search(query)
result[0].page_content

In [ ]:
# FAISS Vector Database

from langchain_community.vectorstores import FAISS

db1 = FAISS.from_documents(documents[:20],OpenAIEmbeddings())

query = "Who are the authors of the attention is all you need research paper?"
result = db1.similarity_search(query)
result[0].page_content

In [ ]:
# Chat Prompt template

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o")
llm

In [ ]:
# Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt)

In [ ]:
# Retriever Chain

# Retriever is an interface to vectore store which has all the information.
# User (question) -> Retriver(interface which has information. In BE it cass vector store) -> LLM(Through Stuff Document chain)

retriever = db.as_retriever()
retriever

In [ ]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [ ]:
response = retrieval_chain.invoke({"input":"What is the formula of scaled dot ?"})
response['answer']